# 2D numerical experiment of the inverted pendulum

In [1]:
import gpytorch
import torch
from src.TVBO import TimeVaryingBOModel
from src.objective_functions_LQR import lqr_objective_function_2D
# from src.prior_mean_model import TimeVaryingBOModelWNNPrior

### Hyperparameters

In [2]:
# parameters regarding the objective function
objective_function_options = {'objective_function': lqr_objective_function_2D,

                              # optimize the last two entries of the feedback gain
                              'spatio_dimensions': 2,

                              # approximate noise level from the objective function
                              'noise_lvl': 0.005,

                              # feasible set for the optimization
                              'feasible_set': torch.tensor([[-62.5, -5], [-12.5, -1]],
                                                           dtype=torch.float),

                              # initial feasible set consisting of only controllers
                              'initial_feasible_set': torch.tensor([[-50, -4], [-25, -2]],
                                                                   dtype=torch.float),

                              # scaling \theta to have approximately equal lengthscales in each dimension
                              'scaling_factors': torch.tensor([3, 1 / 4])}

# parameters regarding the model
model_options = {'constrained_dimensions': None,  # later specified for each variation
                 'forgetting_type': None,  # later specified for each variation
                 'forgetting_factor': None,  # later specified for each variation

                 # specification for the constraints  (cf. Agrell 2019)
                 'nr_samples': 10000,
                 'xv_points_per_dim': 4,  # VOPs per dimensions
                 'truncation_bounds': [0, 2],

                 # specification of prior
                 'prior_mean': 0.,
                 'lengthscale_constraint': gpytorch.constraints.Interval(0.5, 6),
                 'lengthscale_hyperprior': gpytorch.priors.GammaPrior(6, 1 / 0.3),
                 'outputscale_constraint_spatio': gpytorch.constraints.Interval(0, 20),
                 'outputscale_hyperprior_spatio': None, 
                 'n_initial_points': 5}

### Specify variations

In [ ]:
# UI -> UI-TVBO, B2P_OU -> TV-GP-UCB
variations = [
    # OUR APPROACH
    {'forgetting_type': 'UI', 'forgetting_factor': 0.03, 'constrained_dims': [], 'prior_mean': 'DynaBO'},

    # in the paper color blue
    {'forgetting_type': 'UI', 'forgetting_factor': 0.03, 'constrained_dims': [], 'prior_mean': []},

    # in the paper color green
    {'forgetting_type': 'UI', 'forgetting_factor': 0.03, 'constrained_dims': [0, 1], 'prior_mean': []},

    # in the paper color purple
    {'forgetting_type': 'B2P_OU', 'forgetting_factor': 0.03, 'constrained_dims': [0, 1], 'prior_mean': []},

    # in the paper color red
    {'forgetting_type': 'B2P_OU', 'forgetting_factor': 0.03, 'constrained_dims': [], 'prior_mean': []}
]

### Start optimization

In [4]:
trials_per_variation = 25  # number of different runs
for variation in variations:

    # update variation specific parameters
    model_options['forgetting_type'] = variation['forgetting_type']
    model_options['forgetting_factor'] = variation['forgetting_factor']
    model_options['constrained_dimensions'] = variation['constrained_dims']
    n_initial_points = model_options['n_initial_points']
    
    # specify name to safe results
    method_name = model_options['forgetting_type']
    forgetting_factor = model_options['forgetting_factor']
    string = 'constrained' if model_options['constrained_dimensions'] else 'unconstrained'

    NAME = f"{method_name}_2DLQR_{string}_forgetting_factor_{forgetting_factor}_n_initial_points_{n_initial_points}".replace('.', '_')

    if variation['prior_mean'] == 'DynaBO':
        model_options['prior_mean'] = variation['prior_mean']
        NAME = "DynaBOn_" + NAME
        
    tvbo_model = TimeVaryingBOModel(objective_function_options=objective_function_options,
                                    model_options=model_options,
                                    post_processing_options={},
                                    add_noise=False, )  # noise is added during the simulation of the pendulum



    # run optimization
    for trial in range(1, trials_per_variation + 1):
        # tvbo_model.run_TVBO_Wprior(n_initial_points=30,
        #                     time_horizon=300,
        #                     safe_name=NAME,
        #                     trial=trial, )
        tvbo_model.run_TVBO(n_initial_points=n_initial_points,
                            time_horizon=300,
                            safe_name=NAME,
                            trial=trial, )
    print('Finished.')

Initialization


/zfsauton2/home/acapone2/anaconda3/envs/uitvbo/lib/python3.9/site-packages/botorch/fit.py:148: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  warnings.warn(w.message, w.category)


Time for optimizing aquisition functions: 11.175s.

Timestep    5 of Trail  1.
Parameter name: noise value = 0.000
Parameter name: outputscale value = 1.000
Parameter name: lengthscale 0 value = 1.397
Parameter name: lengthscale 1 value = 1.561
Time for optimizing aquisition functions: 0.124s.
Time for optimizing aquisition functions: 0.066s.
Time taken for timestep    5: 0.277s.

Timestep    6 of Trail  1.
Parameter name: noise value = 0.000
Parameter name: outputscale value = 1.000
Parameter name: lengthscale 0 value = 1.516
Parameter name: lengthscale 1 value = 1.789
Time for optimizing aquisition functions: 0.263s.
Time for optimizing aquisition functions: 0.146s.
Time taken for timestep    6: 0.566s.

Timestep    7 of Trail  1.
Parameter name: noise value = 0.000
Parameter name: outputscale value = 1.000
Parameter name: lengthscale 0 value = 1.637
Parameter name: lengthscale 1 value = 1.868
Time for optimizing aquisition functions: 0.148s.
Time for optimizing aquisition functions:

KeyboardInterrupt: 

In [7]:
import os
import pickle
path = './results/'
fxs_ours = []
fxs = []
for i in os.listdir(path):
    if os.path.isfile(os.path.join(path,i)) and 'results_DynaBOn_UI_2DLQR_unconstrained' in i:
        with open('./results/' + i, 'rb') as handle:
            results = pickle.load(handle)
            fxs_ours.append(torch.tensor(results['f_of_x'])*results['inital_data_mean_and_stdv'][1] + results['inital_data_mean_and_stdv'][0])
    if os.path.isfile(os.path.join(path,i)) and 'results_UI_2DLQR_unconstrained' in i:
        with open('./results/' + i, 'rb') as handle:
            results = pickle.load(handle)
            fxs.append(torch.tensor(results['f_of_x'])*results['inital_data_mean_and_stdv'][1] + results['inital_data_mean_and_stdv'][0])
min_shape = min([fx.shape[0] for fx in [fxs,fxs_ours]])
fxs = torch.cat([fx[:min_shape] for fx in fxs])

AttributeError: 'list' object has no attribute 'shape'